# CLIP zero-shot Evaluation
This short notebook implements the dataset split into base and novel categories (see project assignment) and runs the zero-shot evaluation with CLIP.
Feel free to copy the code contained in this notebook or to directly use this notebook as starting point for you project.

In [1]:
# we need to install clip as it is not pre-installed
# you are also free to use open_clip which provide more models
# https://github.com/mlfoundations/open_clip
%pip install openai_clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
  Created wheel for openai_clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=639f9766c36fc108f4a9f20802dd2831386b3766cd1b782cc777df538f6e5b96
  Stored in directory: /root/.cache/pip/wheels/ab/49/bc/c2342e8e14878210ba4825cf314a53f2570f6fb18b91fce3cf
Successfully built openai_clip


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import clip
from tqdm import tqdm
from collections import defaultdict
import random
import numpy as np
from torch.cuda import amp
from torch.utils.data import DataLoader

## Dataset Loading
Let's get the data directly from torchvision as we have seen during labs.

In [3]:
def get_data(data_dir="./data", transform=None):
    """Load Flowers102 train, validation and test sets.
    Args:
        data_dir (str): Directory where the dataset will be stored.
        transform (torch.Compose)
    Returns:
        tuple: A tuple containing the train, validation, and test sets.
    """
    train = torchvision.datasets.Flowers102(root=data_dir, split="train", download=True, transform=transform)
    val = torchvision.datasets.Flowers102(root=data_dir, split="val", download=True, transform=transform)
    test = torchvision.datasets.Flowers102(root=data_dir, split="test", download=True, transform=transform)
    return train, val, test

## Base and Novel categories
To split in base and novel categories we list all dataset classes, and count their number (we already know it's 102 but let's do it properly).
Then, we just allocate the first half to base categories and the remaining half to novel ones.
We can do this because we are simulating a real world application, but keep in mind this will not happen out there!

In [4]:
def base_novel_categories(dataset):
    # set returns the unique set of all dataset classes
    all_classes = set(dataset._labels)
    # and let's count them
    num_classes = len(all_classes)

    # here list(range(num_classes)) returns a list from 0 to num_classes - 1
    # then we slice the list in half and generate base and novel category lists
    base_classes = list(range(num_classes))[:num_classes//2]
    novel_classes = list(range(num_classes))[num_classes//2:]
    return base_classes, novel_classes

## Inspect Classes
Let's now visualize which are the base and novel classes.
To do so, we first get a dummy test set (without augmentations) as we are just interested in the dataset labels. Then, we split it useing `base_novel_categories`.
Finally, we use the hard-coded CLASS_NAMES to print the class in natural language.

> Note: the list of class names was only recently added to `torchvision.datasets.Flowers102`. To avoid useless errors that can occour to you, we decided to also provide such a list.

In [5]:
_, _, tmp_test = get_data()
base_classes, novel_classes = base_novel_categories(tmp_test)
CLASS_NAMES = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", "english marigold", "tiger lily", "moon orchid", "bird of paradise", "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle", "yellow iris", "globe-flower", "purple coneflower", "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", "pincushion flower", "fritillary", "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", "carnation", "garden phlox", "love in the mist", "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip", "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy", "common dandelion", "petunia", "wild pansy", "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia", "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", "black-eyed susan", "silverbush", "californian poppy", "osteospermum", "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory", "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow", "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow", "mexican petunia", "bromelia", "blanket flower", "trumpet creeper", "blackberry lily"]
print("Base Class Names:", [(i, CLASS_NAMES[i]) for i in base_classes])
print("Novel Class Names:", [(i, CLASS_NAMES[i]) for i in novel_classes])

100%|██████████| 345M/345M [00:16<00:00, 20.9MB/s]
100%|██████████| 502/502 [00:00<00:00, 1.78MB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 9.71MB/s]


Base Class Names: [(0, 'pink primrose'), (1, 'hard-leaved pocket orchid'), (2, 'canterbury bells'), (3, 'sweet pea'), (4, 'english marigold'), (5, 'tiger lily'), (6, 'moon orchid'), (7, 'bird of paradise'), (8, 'monkshood'), (9, 'globe thistle'), (10, 'snapdragon'), (11, "colt's foot"), (12, 'king protea'), (13, 'spear thistle'), (14, 'yellow iris'), (15, 'globe-flower'), (16, 'purple coneflower'), (17, 'peruvian lily'), (18, 'balloon flower'), (19, 'giant white arum lily'), (20, 'fire lily'), (21, 'pincushion flower'), (22, 'fritillary'), (23, 'red ginger'), (24, 'grape hyacinth'), (25, 'corn poppy'), (26, 'prince of wales feathers'), (27, 'stemless gentian'), (28, 'artichoke'), (29, 'sweet william'), (30, 'carnation'), (31, 'garden phlox'), (32, 'love in the mist'), (33, 'mexican aster'), (34, 'alpine sea holly'), (35, 'ruby-lipped cattleya'), (36, 'cape flower'), (37, 'great masterwort'), (38, 'siam tulip'), (39, 'lenten rose'), (40, 'barbeton daisy'), (41, 'daffodil'), (42, 'sword 

## Split Dataset
The next step is to actually split the dataset into the base and novel categories we extract from `base_novel_categories`.
To split the data we need the dataset (obviously) and the list of base classes. If the sample label is not part of the base categories, then it must be part of the novel ones.

In [6]:
def split_data(dataset, base_classes):
    # these two lists will store the sample indexes
    base_categories_samples = []
    novel_categories_samples = []

    # we create a set of base classes to compute the test below in O(1)
    # this is optional and can be removed
    base_set = set(base_classes)

    # here we iterate over sample labels and also get the correspondent sample index
    for sample_id, label in enumerate(dataset._labels):
        if label in base_set:
            base_categories_samples.append(sample_id)
        else:
            novel_categories_samples.append(sample_id)

    # here we create the dataset subsets
    # the torch Subset is just a wrapper around the dataset
    # it simply stores the subset indexes and the original dataset (your_subset.dataset)
    # when asking for sample i in the subset, torch will look for its original position in the dataset and retrieve it
    # https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset
    base_dataset = torch.utils.data.Subset(dataset, base_categories_samples)
    novel_dataset = torch.utils.data.Subset(dataset, novel_categories_samples)
    return base_dataset, novel_dataset

## Extract k shots
As the dataset already provides 10 train and validation shots, we do not need to extract them.
Beaware that Few-Shot Adaptation papers must do this operation as most datasets count significantly more samples in both the training and validation sets.

## Load CLIP

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# available models = ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
model, preprocess = clip.load("ViT-B/16", device=device)
model = model.float()  # force the model to use float32

# preprocess contains CLIP's pre-defined augmentations, let's inspect them!
preprocess

100%|███████████████████████████████████████| 335M/335M [00:13<00:00, 26.3MiB/s]


Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x78295c29c360>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

## Load and Prepare Data
Here we get the three dataset split and pass clip pre-defined augmentations.
Then, we compute base and novel categories (in this case is redundand as we already did it before).
Finally, se split the three datasets into base and novel categories.
As we want to use the novel categories only for the test set, we drop `train_novel` and `val_novel`.

In [8]:
# get the three datasets
train_set, val_set, test_set = get_data(transform=preprocess)

# split classes into base and novel
base_classes, novel_classes = base_novel_categories(train_set)

# split the three datasets
train_base, _ = split_data(train_set, base_classes)
val_base, _ = split_data(val_set, base_classes)
test_base, test_novel = split_data(test_set, base_classes)

## Compute Zero-Shot Predictions

In [9]:
@torch.no_grad() # we don't want gradients
def eval(model, dataset, categories, batch_size, device, label=""):
    # let's set the model in evaluation mode
    model.eval()

    # Remap labels into a contiguous set starting from zero
    contig_cat2idx = {cat: idx for idx, cat in enumerate(categories)}

    # here we apply the standard CLIP template used for oxford flowers to all categories
    # and immediately tokenize each sentence (convert natural language into numbers - feel free to print the text input to inspect them)
    text_inputs = clip.tokenize(
        [f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in categories]
    ).to(device)

    # we can encode the text features once as they are shared for all images
    # therefore we do it outside the evaluation loop
    text_features = model.encode_text(text_inputs)
    # and here we normalize them (standard pratice with CLIP)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # simple dataloader creation
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # here we store the number of correct predictions we will make
    correct_predictions = 0
    for image, target in tqdm(dataloader, desc=label):
        # base categories range from 0 to 50, whil novel ones from 51 to 101
        # therefore we must map categories to the [0, 50], otherwise we will have wrong predictions
        # Map targets in contiguous set starting from zero
        # Labels needs to be .long() in pytorch
        target = torch.Tensor([contig_cat2idx[t.item()] for t in target]).long()

        image = image.to(device)
        target = target.to(device)

        # forward image through CLIP image encoder
        image_features = model.encode_image(image)
        # and normalize
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # here cosine similarity between image and text features and keep the argmax for every row (every image)
        predicted_class = (image_features @ text_features.T).argmax(dim=-1)
        # now we check which are correct, and sum them (False == 0, True == 1)
        correct_predictions += (predicted_class == target).sum().item()

    # and now we compute the accuracy
    accuracy = correct_predictions / len(dataset)
    return accuracy

base_accuracy = eval(model=model, dataset=test_base, categories=base_classes, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Base Classes")
novel_accuracy = eval(model=model, dataset=test_novel, categories=novel_classes, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Novel Classes")

print()
print(f"🔍 Base classes accuracy: {base_accuracy*100:.2f}%")
print(f"🔍 Novel classes accuracy: {novel_accuracy*100:.2f}%")


🧠 Zero-shot evaluation on Novel Classes: 100%|██████████| 29/29 [00:44<00:00,  1.54s/it]


🔍 Base classes accuracy: 71.29%
🔍 Novel classes accuracy: 78.24%


## Harmonic Mean
Few-Shot Adaptations papers usually report the Harmonic Mean.
The harmonic mean tends to mitigate the impact of large outliers (base accuracy) and aggravate the impact of small ones (novel accuracy).
Thus, achieving very high base accuracies at the expense of the novel accuracy will be penalized by the HM.

In [10]:
def harmonic_mean(base_accuracy, novel_accuracy):
    numerator = 2
    denominator = 1 / base_accuracy + 1 / novel_accuracy
    hm = numerator / denominator
    return hm

print(f"🔍 Harmonic Mean: {harmonic_mean(base_accuracy, novel_accuracy)*100:.2f}%")

🔍 Harmonic Mean: 74.60%


## Introduction

Large-scale vision–language models such as **CLIP** achieve remarkable *zero-shot* recognition by embedding images and text into a shared semantic space.  

However, in **few-shot adaptation**—where base classes $\mathcal{C}_b$ have labeled examples and novel classes $\mathcal{C}_n$ are known only by name—CLIP remains unbalanced: it performs strongly on novel classes (thanks to its text priors) but struggles to fully exploit supervision on base classes.  

We introduce a **Coupled Curved Text-to-Text Dictionary Transport (C2T2D)** framework that learns a **self-contained transport field** on CLIP’s **text manifold**, without relying on a zero-shot teacher.  
The goal is to let each class anchor *adapt geometrically* to image evidence while preserving CLIP’s intrinsic structure.

---

- On the **text side**, we model CLIP’s embedding space as a **curved manifold**.  
  Around each canonical class anchor $z_c$, we build a **tangent dictionary** $G_c$ by *mixing shared atlas dictionaries* through class-conditioned attention.  
  Each $G_c$ spans valid semantic directions on the unit sphere—directions along which the meaning of a class can move without leaving CLIP’s manifold.

- On the **image side**, the transport head predicts how each image embedding $x$ should **move its corresponding anchor** along those directions using a small **geodesic update**:
  $$
  z_c' = z_c \cos s_c + u_c \sin s_c,
  $$
  where $u_c$ is a predicted tangent direction and $s_c$ is a learned step size that controls the rotation magnitude.  
  This keeps the motion smooth, localized, and consistent with CLIP’s geometry.

- The head is trained **entirely from data**, without zero-shot distillation, using a **contrastive (InfoNCE)** objective combined with light geometric regularizers (consistency, span, and step penalties).  
  These ensure stable, geometry-preserving transport that respects the structure of CLIP’s text manifold.

---

By learning to align images and class anchors through **geodesic motion** rather than fixed zero-shot templates,  
C2T2D achieves a balanced improvement across **base** and **novel** classes.  
The model maintains CLIP’s zero-shot robustness while introducing a *teacher-free, manifold-aware refinement* that optimizes the **harmonic mean (HM)** of both domains.

## Phrase Bank and Text Encoders

To capture linguistic variability, we build a **phrase bank** of diverse textual prompts for each class, such as  
*“a close-up of {name}”*, *“a photo of blooming {name}”*, or *“a natural photo of {name}”*.  
This exposes CLIP’s text encoder to a broad range of linguistic contexts rather than relying on a single rigid template.  

Each phrase is encoded with CLIP’s text encoder $g_\phi(\cdot)$:

$$
z_{c,j} = g_\phi(t_{c,j}), \qquad
\hat{z}_{c,j} = \frac{z_{c,j}}{\|z_{c,j}\|},
$$

where $t_{c,j}$ is the $j$-th textual variant for class $c$.  
The normalized embeddings $\hat{z}_{c,j}$ form a *cloud* of samples that locally describe the same concept under different phrasings.  

From this cloud we define a **canonical class anchor** using a neutral template:

$$
z_c = g_\phi(\text{“a photo of a } c \text{, a type of flower”}), \qquad \|z_c\| = 1.
$$

---

### Building a shared semantic atlas
To promote cross-class sharing of geometric structure, we aggregate *all* phrase embeddings across classes and cluster them into $M$ **semantic hubs** with centers $\{a_m\}_{m=1}^M$.  
Each hub defines an **atlas anchor** $a_m$ and a local dictionary $G_m$ that captures common directions of variation (e.g., “macro shot,” “in a garden,” “color shift”).  

Each class then forms its own **tangent basis** by softly mixing these shared dictionaries through class-conditioned attention:

$$
\alpha_{c,m} = \frac{\exp(z_c^\top a_m / \tau)}{\sum_{m'} \exp(z_c^\top a_{m'} / \tau)}, \qquad
\hat{G}_c = \sum_{m=1}^M \alpha_{c,m}\, \mathrm{Proj}_{T_{z_c}}(G_m),
$$

followed by QR re-orthonormalization to ensure orthogonality on the sphere.  
The resulting $\hat{G}_c$ serves as the **final tangent basis** for class $c$, combining both *local semantics* and *global manifold structure*.

---

**Intuition:**  
- The phrase variants $\hat{z}_{c,j}$ capture *semantic diversity* for each class.  
- The atlas hubs $\{a_m, G_m\}$ capture *global patterns* of meaning shared across classes.  
- The mixed tangent basis $\hat{G}_c$ integrates both, defining *how the class concept can bend or vary* while staying faithful to CLIP’s manifold geometry.  

This hybrid geometric representation allows the model to reason about **directions of meaning** on the manifold — enabling smooth, manifold-aware adaptation during training.

In [11]:
def build_phrase_variants(class_names, max_per_class=200, seed=0):
    random.seed(seed)
    TEMPLATES = [
        "a {name}",
        "a close-up of {name}",
        "a macro photo of {name}",
        "a photo of {name} petals",
        "a photo of blooming {name}",
        "a photo of {name} in a garden",
        "a botanical photo of {name}",
        "a natural photo of {name}",
        "a {name} flower",
        "a {name} blossom",
    ]
    phrases = {c: [] for c in class_names}
    for c in class_names:
        extras = [
            f"vibrant {c}", f"wild {c}", f"fresh {c}", f"single {c}", f"double {c}",
            f"pink {c}", f"yellow {c}", f"white {c}", f"red {c}", f"purple {c}"
        ]
        pool = TEMPLATES + [t.replace("{name}", e) for e in extras for t in ["a photo of {name}"]]
        gen = []
        for _ in range(max_per_class):
            t = random.choice(pool)
            gen.append(t.format(name=c))
        phrases[c] = list(dict.fromkeys(gen))[:max_per_class]
    return phrases

def _tokenize_batch(text_list, device):
    return clip.tokenize(text_list).to(device)

def encode_text_prompts(text_list, device):
    tokenized = _tokenize_batch(text_list, device)
    with torch.no_grad():
        z = model.encode_text(tokenized)
        z = z / z.norm(dim=-1, keepdim=True)
    return z

@torch.no_grad()
def class_name_embeddings(class_names, device):
    texts = [f"a photo of a {c}, a type of flower." for c in class_names]
    Z = encode_text_prompts(texts, device)
    return Z

## Tangent Space Utilities

Since CLIP text embeddings are L2-normalized, all text features lie on the **unit hypersphere**  
$S^{d-1} \subset \mathbb{R}^d$.  
Linear operations in $\mathbb{R}^d$ can distort this curved geometry, so to reason about local structure correctly,  
we work in **tangent spaces** around each canonical text anchor $z_c$.

---

### Tangent space definition
For a unit anchor $z_c$, its tangent space is
$$
T_{z_c} = \{\, v \in \mathbb{R}^d \;|\; v^\top z_c = 0 \,\}.
$$

This space contains all directions orthogonal to $z_c$, corresponding to infinitesimal “moves” that stay on the surface of CLIP’s sphere.  
It provides a local linearization of the text manifold near that concept.

---

### From phrase variants to local directions
To characterize how a class concept can *vary semantically*,  
we generate multiple text prompts describing the same class (e.g.,  
“a photo of a rose,” “a close-up of a rose,” “a rose in a garden”).  
Each prompt embedding $\hat{z}_{c,j}$ is L2-normalized and projected onto $T_{z_c}$:

$$
\tilde{z}_{c,j} = \hat{z}_{c,j} - (\hat{z}_{c,j}^\top z_c)\, z_c.
$$

Collecting all projected variants $\tilde{z}_{c,j}$, we perform SVD:

$$
\tilde{Z}_c = U\,\Sigma\,V^\top, \quad G_c = V_{1:r}.
$$

The columns of $G_c \in \mathbb{R}^{d \times r}$ form an **orthonormal local basis** of semantic variation around $z_c$.

---

### Building a shared semantic atlas
While each $G_c$ describes local variation for its class, these directions are often redundant across classes  
(e.g., “macro view,” “in a garden,” “color tone”).  
To share structure, we cluster all prompt embeddings from every class into $M$ **semantic hubs** with centers $\{a_m\}_{m=1}^M$,  
each defining an **atlas anchor** and its own local dictionary $G_m$.

Every class then obtains its final tangent basis $\hat{G}_c$ by **mixing** these shared atlas dictionaries through class-conditioned attention:

$$
\alpha_{c,m} = \frac{\exp(z_c^\top a_m / \tau)}{\sum_{m'} \exp(z_c^\top a_{m'} / \tau)}, \qquad
\hat{G}_c = \mathrm{QR}\!\left(\sum_{m=1}^M \alpha_{c,m}\, \mathrm{Proj}_{T_{z_c}}(G_m)\right),
$$

where the projection ensures each direction lies in $T_{z_c}$ and the QR step re-orthonormalizes them.  
These $\hat{G}_c$ are the **final text-side bases** used by the transport head.

---

### Intuition
- Each class anchor $z_c$ lies on CLIP’s unit sphere and represents a concept.  
- Its mixed tangent basis $\hat{G}_c$ provides valid *semantic directions* of motion, combining local prompt structure and shared atlas geometry.  
- Restricting movement to these directions lets the model deform meaning smoothly and geometrically—  
  preserving CLIP’s manifold structure while allowing flexible adaptation.

This hierarchical construction makes the learned transport **manifold-aware** and **semantically shared**:  
local per-class semantics align with global variation patterns captured across the atlas.

In [12]:
@torch.no_grad()
def build_tangent_bases(class_names, device, anchors_per_class=200, r=12, seed=0):
    phrases = build_phrase_variants(class_names, anchors_per_class, seed)
    Z0 = class_name_embeddings(class_names, device)
    bases = []
    for i, cname in enumerate(class_names):
        Zc = encode_text_prompts(phrases[cname], device)
        z0 = Z0[i:i+1]
        # project anchors to tangent plane at z0
        Zc_tan = Zc - (Zc @ z0.T) * z0
        U, S, Vh = torch.linalg.svd(Zc_tan, full_matrices=False)
        T = Vh[:r].T.contiguous()
        T = T - z0.T @ (z0 @ T)
        Q, _ = torch.linalg.qr(T, mode='reduced')
        bases.append(Q)
    return Z0, torch.stack(bases, dim=0)

@torch.no_grad()
def _torch_kmeans(X, M=3, iters=15, seed=0):
    """Simple k-means clustering on L2-normalized embeddings (works on CPU/CUDA)."""
    g = torch.Generator(device=X.device).manual_seed(42 + int(seed))
    N = X.size(0)
    assert N >= M, f"k-means needs N(={N}) >= M(={M})"

    init_idx = torch.randint(0, N, (M,), generator=g, device=X.device)
    centers = X[init_idx].clone()

    for _ in range(iters):
        # cosine k-means (since X is L2-normalized)
        assign = (X @ centers.T).argmax(dim=1)
        new_centers = []
        for m in range(M):
            mask = (assign == m)
            if mask.any():
                c = X[mask].mean(dim=0, keepdim=True)
            else:
                ridx = torch.randint(0, N, (1,), generator=g, device=X.device)
                c = X[ridx]
            c = c / c.norm(dim=-1, keepdim=True).clamp_min(1e-8)
            new_centers.append(c)

        centers = torch.cat(new_centers, dim=0)
    return centers / centers.norm(dim=-1, keepdim=True).clamp_min(1e-8)

@torch.no_grad()
def build_text_anchors_and_bank(class_names_all, anchors_per_class=400, K=12, M=3, device="cpu", seed=0):
    """Cluster text embeddings into atlas anchors + per-cluster dictionaries"""
    phrases_all = build_phrase_variants(class_names_all, anchors_per_class, seed)
    Z_all = encode_text_prompts([t for c in class_names_all for t in phrases_all[c]], device)
    A = _torch_kmeans(Z_all, M=M, iters=20, seed=seed)
    assign = (Z_all @ A.T).argmax(dim=1)
    G_bank = []
    for m in range(M):
        Zm = Z_all[assign == m] if (assign == m).any() else Z_all
        U, S, Vh = torch.linalg.svd(Zm - Zm.mean(dim=0, keepdim=True), full_matrices=False)
        Q, _ = torch.linalg.qr(Vh[:K].T.contiguous(), mode='reduced')
        G_bank.append(Q)
    return A, torch.stack(G_bank, dim=0)

@torch.no_grad()
def class_tangent_mixture(Z0, A_centers, G_bank, tau=0.07):
    """Mix atlas dictionaries per class to form tangent dictionary (batched QR)."""
    C, d = Z0.shape
    M, _, K = G_bank.shape
    att = (Z0 @ A_centers.T) / tau
    alpha = torch.softmax(att, dim=-1)

    I = torch.eye(d, device=Z0.device).unsqueeze(0)
    Proj = I - Z0.unsqueeze(-1) @ Z0.unsqueeze(1)

    # project each bank Gm to each class tangent space in batch
    Gc_all = []
    for m in range(M):
        Gc = torch.einsum('cij,jk->cik', Proj, G_bank[m])
        Q, _ = torch.linalg.qr(Gc, mode='reduced')
        Gc_all.append(Q)
    Gc_all = torch.stack(Gc_all, dim=1)

    # mix by attention weights, then re-orthonormalize once
    Gc_mix = (alpha.view(C, M, 1, 1) * Gc_all).sum(dim=1)
    Q_mix, _ = torch.linalg.qr(Gc_mix, mode='reduced')
    return Q_mix, alpha

## Image Prototypes

While tangent dictionaries capture **semantic variation** from text prompts, they may not perfectly reflect how objects look in the dataset.  
To complement them, we compute **image prototypes** from the few available training samples in the base classes.

---

#### Computing prototypes
For each base class $c \in \mathcal{C}_b$, we encode its few-shot images with CLIP’s image encoder $f_\theta$:

$$
u_i = f_\theta(x_i), \quad \hat{u}_i = \frac{u_i}{\|u_i\|}.
$$

We then cluster the embeddings $\hat{u}_i$ belonging to class $c$ into $k$ groups and use their centers as **class prototypes**:

$$
p_{c,m} \approx \text{cluster center of } \{ \hat{u}_i : y_i = c \}, \quad m = 1, \dots, k.
$$

Each $p_{c,m}$ is L2-normalized.

---

#### Projection into tangent space
To compare prototypes with text anchors, we project each prototype into the tangent space at the canonical anchor $z_c$:

$$
q_{c,m} = p_{c,m} - (p_{c,m}^\top z_c) z_c, \quad
\hat{q}_{c,m} = \frac{q_{c,m}}{\|q_{c,m}\|}.
$$

The set

$$
Q_c = \{ \hat{q}_{c,1}, \dots, \hat{q}_{c,k} \}
$$

thus forms **prototype-guided tangent directions** for class $c$.

---

#### Intuition
- Text dictionaries $G_c$ provide **semantic directions** from linguistic variation (different ways humans describe the class).  
- Prototypes $Q_c$ provide **visual directions** grounded in the actual data distribution.  
- By projecting prototypes into the tangent at $z_c$, we ensure that the visual evidence is expressed as *local semantic variations* around the anchor.  

Together, $G_c$ and $Q_c$ enrich the representation of each class, letting the model align **what we say about a class** (text) with **how it actually looks** (images).

In [13]:
# @torch.no_grad()
# def class_multi_protos(train_subset, class_ids, clip_model, device, k=2, iters=10, seed=0):
#     """Compute k-means prototypes per class from few-shot set"""
#     cont = {c:i for i,c in enumerate(class_ids)}
#     feats_by_c = [[] for _ in class_ids]
#     loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=False, num_workers=2)
#     for imgs, labels in loader:
#         x = clip_model.encode_image(imgs.to(device))
#         x = x / x.norm(dim=-1, keepdim=True)
#         for i, lab in enumerate(labels.tolist()):
#             if lab in cont:
#                 feats_by_c[cont[lab]].append(x[i].detach())
#     Pk_list = []
#     for xs in feats_by_c:
#         if not xs:
#             Pk_list.append(None); continue
#         X = torch.stack(xs, 0)
#         C = X[torch.randperm(X.size(0), device=device)[:k]]
#         for _ in range(iters):
#             assign = (X @ C.T).argmax(dim=1)
#             newC = []
#             for m in range(k):
#                 xm = X[assign == m] if (assign == m).any() else X
#                 c = xm.mean(0, keepdim=True)
#                 newC.append(c / c.norm(dim=-1, keepdim=True))
#             C = torch.cat(newC, dim=0)
#         Pk_list.append(C)
#     return Pk_list

# @torch.no_grad()
# def build_Q_multi(Z0, Pk_list, k):
#     """Build tangent directions from prototypes toward text anchors"""
#     C, d = Z0.shape
#     Qk = torch.zeros(C, k, d, device=Z0.device)
#     mask = torch.zeros(C, dtype=torch.bool, device=Z0.device)
#     for j in range(C):
#         if Pk_list[j] is None: continue
#         P = Pk_list[j]
#         z = Z0[j:j+1]
#         q = P - (P * z).sum(dim=-1, keepdim=True) * z
#         q = q / (q.norm(dim=-1, keepdim=True).clamp_min(1e-8))

#         Qk[j, :q.size(0)] = q; mask[j] = True
#     return Qk, mask

## Model Head

Now that we have:
- **Canonical text anchors** $z_c \in \mathbb{R}^d$ (one per class),
- **Mixed tangent dictionaries** $G_c \in \mathbb{R}^{d \times K}$, obtained by combining shared atlas directions specific to each class,

we introduce the **C2T2D Transport Head** —  
a lightweight module that predicts, for each image–class pair, a small **tangent-space displacement** and a **step size** describing how far to move along CLIP’s text manifold.  
This produces a *transported version* of each class anchor that better aligns with the image while remaining on the unit sphere.

---

### Input features
For every image embedding $x \in \mathbb{R}^d$ and class anchor $z_c$:

- **Cosine similarity:** $s = x^\top z_c$
- **Chordal distance:** $1 - s$

We concatenate these with both embeddings to form the feature vector:

$$
h_{x,c} = [x,\, z_c,\, s,\, 1-s] \in \mathbb{R}^{2d + 2}.
$$

This representation captures both semantic and geometric context for each image–class pair.

---

### 1. Predicting the tangent-space direction
A multilayer perceptron (MLP) takes $h_{x,c}$ and predicts coefficients $\beta_{c,k}$ over the $K$ tangent directions of $G_c$:

$$
\beta_c = \text{MLP}_t(h_{x,c}) \in \mathbb{R}^K.
$$

These coefficients combine the tangent basis vectors $G_c = [g_{c,1}, \dots, g_{c,K}]$ into a single **semantic motion**:

$$
u_c'(x) = \sum_{k=1}^K \beta_{c,k}\, g_{c,k}.
$$

To ensure the motion stays on the sphere, we project it onto the tangent space of $z_c$ and re-normalize:

$$
u_c(x) = \frac{u_c'(x) - (u_c'(x)^\top z_c)\,z_c}{\|\,u_c'(x) - (u_c'(x)^\top z_c)\,z_c\|}.
$$

This gives a valid unit direction $u_c(x) \in T_{z_c}$ —  
the direction in which the class anchor should move to align with $x$.

---

### 2. Predicting the step size
A second MLP predicts how far to move along this direction:

$$
s_c(x) = \varepsilon_t \cdot \mathrm{softplus}\!\big(\text{MLP}_s(h_{x,c})\big), \qquad \varepsilon_t = 0.25.
$$

The softplus ensures positivity, and the cap $\varepsilon_t$ keeps motion small and stable.  
Unlike earlier versions, this step is **not similarity-gated** — the model learns its own step magnitude per pair.

---

### 3. Geodesic update on the unit sphere
Using $s_c(x)$ and $u_c(x)$, we perform a **geodesic move** — the natural rotation that keeps points on the sphere:

$$
\tilde{z}_c(x) = \cos(s_c)\, z_c + \sin(s_c)\, u_c(x),
\qquad \|\tilde{z}_c(x)\| = 1.
$$

This produces a **transported anchor** $\tilde{z}_c(x)$ that remains normalized and geometrically valid.

---

### 4. Classification logits
We score each class by its similarity to the image after transport:

$$
\ell_c(x) = (x^\top \tilde{z}_c(x)) \cdot \exp(\gamma),
$$

where $\gamma$ is CLIP’s learned logit scale.  
These logits feed into a **contrastive (InfoNCE)** loss during training.

---

### Intuition
- The **mixed tangent bases** $G_c$ capture how a class’s meaning can bend locally within the shared atlas of CLIP’s text manifold.  
- The **transport head** predicts *which way to bend* (via $\beta_c$) and *how far* (via $s_c$).  
- The **geodesic move** preserves the manifold geometry, ensuring the anchor remains valid.  
- The process refines text–image alignment without altering CLIP’s overall structure, maintaining its zero-shot generalization.

In essence, the head learns a **local, geometry-preserving navigation rule**:  
for each image, move each text anchor slightly in the right direction — always staying on the manifold.

In [14]:
class C2T2D_Head(nn.Module):
    """
    Teacher-free geodesic transport head:
      - predicts basis mix beta (over Gc)
      - predicts step size s >= 0
      - outputs transported anchors W and logits against x_img
      - returns (logits, W, u, s, beta) for regularizers
    """
    def __init__(self, d, K_text, Cb=0):
        super().__init__()
        self.bias_c = nn.Parameter(torch.zeros(Cb)) if Cb > 0 else None
        in_dim = 2*d + 2

        self.dir_mlp = nn.Sequential(
            nn.Linear(in_dim, 256), nn.ReLU(), nn.Dropout(0.10),
            nn.Linear(256, 256), nn.ReLU(), nn.Dropout(0.10),
            nn.Linear(256, K_text)
        )
        self.step_mlp = nn.Sequential(
            nn.Linear(in_dim, 128), nn.ReLU(),
            nn.Linear(128, 1)  # will softplus -> [0, +)
        )

    def forward(self, x_img, Z0, Gc):
        """
        x_img: [B,d] normalized
        Z0:    [C,d] class anchors (normalized)
        Gc:    [C,d,K] orthonormal tangent bases per class
        """
        B, d = x_img.shape
        C, d2, K = Gc.shape
        assert d == d2

        # tile tensors
        X = x_img.unsqueeze(1).expand(B, C, d)
        Z = Z0.unsqueeze(0).expand(B, C, d)

        # simple image-text features for the head
        xdot = (X * Z).sum(-1, keepdim=True)
        dist2 = 1.0 - xdot
        h_in = torch.cat([X, Z, xdot, dist2], dim=-1)

        # predict basis mix and step
        beta = self.dir_mlp(h_in)
        s = F.softplus(self.step_mlp(h_in))
        s = 0.25 * s

        # build unit tangent direction u from class basis
        GcBC = Gc.unsqueeze(0).expand(B, -1, -1, -1)
        u = (beta.unsqueeze(-2) @ GcBC.transpose(-2, -1)).squeeze(-2)
        u = u - (u * Z).sum(-1, keepdim=True) * Z
        u = u / (u.norm(dim=-1, keepdim=True).clamp_min(1e-8))

        # geodesic exponential map on the unit sphere
        zcos = torch.cos(s)
        zsin = torch.sin(s)
        Zp = Z * zcos + u * zsin
        W = F.normalize(Zp, dim=-1)

        # logits against images
        logits = torch.einsum('bd,bcd->bc', x_img, W) * model.logit_scale.exp()
        if self.bias_c is not None:
            logits[:, :self.bias_c.numel()] += self.bias_c

        return logits, W, u, s, beta

#### Why This Classifier Works

The classifier performs **manifold-constrained alignment** between image and text embeddings.  
Rather than arbitrarily warping CLIP’s space, it learns to move **along** the intrinsic geometry of the unit hypersphere —  
adjusting class anchors through small, valid rotations within their tangent spaces.

This operation admits two equivalent geometric interpretations:

- **Image-side view:** the image embedding $x$ is *transported* toward its most compatible class anchor $z_c$  
  along directions that are valid within CLIP’s text manifold.
- **Text-side view (used in practice):** the class anchor $z_c$ is *locally transported* toward the image inside its own tangent space.

Both describe the **same geodesic rotation** on the sphere; only the frame of reference differs.

---

If $x$ **belongs to class $c$**, the model predicts a small motion that follows the low-curvature structure around $z_c$,  
bringing the transported anchor $\tilde{z}_c(x)$ closer to $x$ and producing a high cosine score.

If $x$ **does not belong to class $c$**, any movement allowed by the tangent-space basis fails to align with $x$,  
since the manifold itself restricts how far the anchor can bend — the similarity therefore remains low.

Thus, instead of asking:

$$
\text{“Which fixed anchor is closest to } x \text{?”}
$$

the classifier now asks:

$$
\text{“Which class anchor can align with } x \text{through a valid geodesic move on the manifold?”}
$$

---

#### Why This Helps

- The model is **manifold-aware** — every update is a geodesic motion that preserves CLIP’s spherical structure.  
- Each anchor **adapts semantically** through its tangent directions without leaving the manifold.  
- **Base and novel classes** share this same geometric mechanism, enabling consistent generalization.  
- The system balances **semantic flexibility** (moving along valid directions) and **geometric stability** (staying on the sphere).

---

In short:  
classification becomes a search for the **most geometrically plausible alignment** between an image and each class anchor —  
where all motion is confined to **legitimate directions of meaning** on CLIP’s text manifold.

## Training Helpers

In [15]:
def few_shot_subset(dataset, base_class_indices, shots=10, seed=0):
    rng = random.Random(seed)
    indices_by_class = defaultdict(list)
    if isinstance(dataset, torch.utils.data.Subset):
        for i, orig_idx in enumerate(dataset.indices):
            label = dataset.dataset._labels[orig_idx]
            if label in base_class_indices:
                indices_by_class[label].append(orig_idx)
    else:
        for i, label in enumerate(dataset._labels):
            if label in base_class_indices:
                indices_by_class[label].append(i)

    selected = []
    for c in base_class_indices:
        pool = indices_by_class[c]
        rng.shuffle(pool)
        selected.extend(pool[:shots])

    return torch.utils.data.Subset(
        dataset.dataset if isinstance(dataset, torch.utils.data.Subset) else dataset,
        selected
    )

@torch.no_grad()
def eval_kpb(head, class_ids, Z0, Gc, Qk, dataset, desc, T_star=1.0):
    loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False, num_workers=2)
    cont = {orig:i for i, orig in enumerate(class_ids)}
    ok = tot = 0
    for imgs, lbls in tqdm(loader, desc=desc):
        imgs = imgs.to(device)
        y = torch.tensor([cont[l.item()] for l in lbls], device=device)
        x = model.encode_image(imgs); x = x / x.norm(dim=-1, keepdim=True)
        logits, _, _, _, _ = head(x, Z0, Gc)
        pred = (logits / T_star).argmax(1)
        ok += (pred == y).sum().item(); tot += imgs.size(0)
    return ok / max(1, tot)

## Data preparation

In [16]:
torch.manual_seed(42); np.random.seed(42); random.seed(42)

base_names = [CLASS_NAMES[i] for i in base_classes]
novel_names = [CLASS_NAMES[i] for i in novel_classes]
A_centers, G_bank = build_text_anchors_and_bank(base_names, anchors_per_class=400, K=20, M=2, device=device)
Z0_base = class_name_embeddings(base_names,  device)
Z0_novel = class_name_embeddings(novel_names, device)
Gc_base_mix, _ = class_tangent_mixture(Z0_base,  A_centers, G_bank, tau=0.07)
Gc_novel_mix, _ = class_tangent_mixture(Z0_novel, A_centers, G_bank, tau=0.07)

# build few-shot subset for base
train_base_full, _ = split_data(train_set, base_classes)
train_10shot = few_shot_subset(train_base_full, base_classes, shots=10, seed=42)
g = torch.Generator().manual_seed(42)
train_loader = torch.utils.data.DataLoader(
    train_10shot,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    generator=g
)

# base prototypes
# K_PROTO = 3
# Pk_base = class_multi_protos(train_10shot, base_classes, model, device, k=K_PROTO, iters=10)
# Qk_base, _ = build_Q_multi(Z0_base, Pk_base, k=K_PROTO)
# Qk_novel = torch.zeros(Z0_novel.size(0), K_PROTO, Z0_novel.size(1), device=device)

## Compute Few-Shot (FSA) Predictions

Once the model architecture is in place, we train the transport head using a mix of **contrastive, geometric, and stabilization losses**.  
The goal is to learn a smooth, manifold-aware transport field that improves alignment on base classes while preserving CLIP’s zero-shot structure for novel ones.

---

#### 1. Contrastive Cross-Entropy (InfoNCE)
For each batch, we compute logits $\ell_c(x)$ between image embeddings $x$ and their transported text anchors $\tilde{z}_c(x)$.  
As in CLIP, we apply a temperature-scaled cross-entropy loss:

$$
\mathcal{L}_{ctr} = \text{CrossEntropy}\!\left( \frac{\ell(x)}{T_{\text{InfoNCE}}},\, y \right).
$$

This is the **primary training signal**, encouraging each image to align most strongly with its correct class after geodesic transport.

---

#### 2. Dropout Consistency
To promote stability, the head is evaluated twice per batch with dropout active, producing two sets of transported anchors $W$ and $W'$.  
We enforce consistency between them:

$$
\mathcal{L}_{cons} = \big(1 - \cos(W, W')\big)_{\text{mean}}.
$$

This regularizer reduces stochastic variance and encourages smooth atlas mixtures across dropout perturbations.

---

#### 3. Span Regularization
To ensure the predicted motion $u_c(x)$ stays within each class’s valid tangent subspace (spanned by its mixed basis $G_c$),  
we penalize any component that lies outside that subspace:

$$
\mathcal{L}_{span} = \|u - P_{G_c}u\|_2^2,
$$

where $P_{G_c} = G_c G_c^\top$ is the projection matrix computed from the current bases.  
This keeps the learned transport **geometry-consistent**, ensuring movement occurs only along valid manifold directions.

---

#### 4. Small-Step Prior
To maintain local linearity and avoid overfitting, we penalize large step sizes:

$$
\mathcal{L}_{step} = \|s\|_2^2.
$$

This term encourages smooth, interpretable rotations on the sphere rather than abrupt reorientations.

---

#### 5. Weak Zero-Shot Tether
Although the head trains without any zero-shot teacher, we add a light **hinge tether** that prevents transported logits from dropping too far below CLIP’s original zero-shot scores:

$$
\mathcal{L}_{tether} = \max\!\big(0,\, (\ell_{\text{ZS}} - \ell) - \delta\big),
$$

where $\delta$ is a small safety margin (e.g., 0.02).  
This term does *not* introduce gradients from CLIP itself—it simply prevents the model from degrading its prior knowledge.

---

#### 6. Total Loss
The final training objective is a weighted combination:

$$
\begin{aligned}
\mathcal{L} =&\;
\mathcal{L}_{ctr}
+ 0.3\, \mathcal{L}_{cons}
+ 0.1\, \mathcal{L}_{span}
+ 0.05\, \mathcal{L}_{step} \\
&+ 0.05\, \mathcal{L}_{tether}.
\end{aligned}
$$

These coefficients are empirically tuned for stability and transfer balance.

---

#### Intuition
- **Contrastive loss** enforces correct alignment.  
- **Consistency** stabilizes stochastic behavior.  
- **Span** and **step** regularizers keep the motion geometrically valid and small.  
- The **ZS tether** prevents catastrophic forgetting of CLIP’s zero-shot prior.

Together, these losses teach the head to learn a **smooth, geometry-preserving transport field**—  
refining base classes through data-driven alignment while generalizing gracefully to unseen novel ones.

In [17]:
use_cuda = str(device).startswith("cuda")

d = Z0_base.size(1); K_text = Gc_base_mix.size(2)
head = C2T2D_Head(d, K_text).to(device)

EPOCHS = 20
opt = torch.optim.AdamW(head.parameters(), lr=1e-3, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    opt, max_lr=2e-3, steps_per_epoch=len(train_loader), epochs=EPOCHS, pct_start=0.3
)

T_INFONCE = 0.10
contig = {orig: i for i, orig in enumerate(base_classes)}
best_hm, best_state = -1.0, None

for epoch in range(EPOCHS):
    head.train(); tot = n = 0
    for images, labels in train_loader:
        images = images.to(device)
        y = torch.tensor([contig[l.item()] for l in labels], device=device)

        with torch.no_grad():
            with torch.amp.autocast(device_type="cuda", enabled=use_cuda, dtype=torch.float16):
                x = model.encode_image(images)
            x = F.normalize(x.float(), dim=-1)

        # Forward: teacher-free transport
        logits, W, u, s, beta = head(x, Z0_base, Gc_base_mix)

        # -------- Losses ----------
        # 1) InfoNCE (image -> transported text), classic CE over classes
        loss_ctr = F.cross_entropy(logits / T_INFONCE, y)

        # 2) Simple dropout-consistency: run the head twice and align W
        if head.training:
            logits2, W2, _, _, _ = head(x, Z0_base, Gc_base_mix)
            loss_cons = (1.0 - (W * W2).sum(dim=-1)).mean()  # 1 - cosine since W are unit
        else:
            loss_cons = torch.tensor(0.0, device=device)

        # 3) Span regularizer built from the CURRENT Gc used in forward
        Gc_used = Gc_base_mix
        PB = (Gc_used @ Gc_used.transpose(1, 2))
        PB = PB.unsqueeze(0).expand(x.size(0), -1, -1, -1)
        u_proj = (PB @ u.unsqueeze(-1)).squeeze(-1)
        loss_span = ((u - u_proj)**2).mean()

        # 4) Small-step prior
        loss_step = (s**2).mean()

        # 5) Weak ZS tether (hinge so we don't tank below ZS by a lot)
        with torch.no_grad():
            zs_logits = (x @ Z0_base.T) * model.logit_scale.exp()
        margin = 0.02
        loss_tether = F.relu(zs_logits - logits - margin).mean()

        # Weights (these are educated guesses, need finetuning)
        loss = (
            loss_ctr
            + 0.3 * loss_cons
            + 0.1 * loss_span
            + 0.05 * loss_step
            + 0.05 * loss_tether
        )
        # --------------------------

        opt.zero_grad(); loss.backward()
        torch.nn.utils.clip_grad_norm_(head.parameters(), 1.0)
        opt.step()
        scheduler.step()

        tot += loss.item() * images.size(0); n += images.size(0)

    print(f"Epoch {epoch+1}: loss={(tot/n):.4f}")

# eval + checkpoint
head.eval()
b = eval_kpb(head, base_classes,  Z0_base,  Gc_base_mix,  None, test_base,  "C2T2D Base")
nv = eval_kpb(head, novel_classes, Z0_novel, Gc_novel_mix, None, test_novel, "C2T2D Novel")
hm = 2 / (1 / b + 1 / nv)
print(f"[C2T2D] Base={b*100:.2f}% Novel={nv*100:.2f}% HM={hm*100:.2f}%")
if hm > best_hm:
    best_hm, best_state = hm, {k: v.detach().cpu().clone() for k, v in head.state_dict().items()}
    print("   -> saved best C2T2D-HM checkpoint")

Epoch 1: loss=9.1639
Epoch 2: loss=8.2000
Epoch 3: loss=6.7442
Epoch 4: loss=4.1904
Epoch 5: loss=2.5762
Epoch 6: loss=1.6945
Epoch 7: loss=1.5890
Epoch 8: loss=0.9780
Epoch 9: loss=0.6563
Epoch 10: loss=0.5348
Epoch 11: loss=0.4239
Epoch 12: loss=0.3910
Epoch 13: loss=0.2394
Epoch 14: loss=0.1960
Epoch 15: loss=0.1441
Epoch 16: loss=0.0707
Epoch 17: loss=0.1120
Epoch 18: loss=0.0557
Epoch 19: loss=0.0627
Epoch 20: loss=0.0471


C2T2D Novel: 100%|██████████| 29/29 [00:43<00:00,  1.51s/it]

[C2T2D] Base=90.17% Novel=76.82% HM=82.96%
   -> saved best C2T2D-HM checkpoint


## Final Evaluation

In [18]:
@torch.no_grad()
def _encode_imgs(imgs, device):
    with torch.amp.autocast(device_type="cuda", enabled=str(device).startswith("cuda"), dtype=torch.float16):
        x = model.encode_image(imgs.to(device))
    return F.normalize(x.float(), dim=-1)

@torch.no_grad()
def eval_zs(dataset, class_ids, Z0, batch=128, name="ZS", device="cuda"):
    cont = {orig:i for i, orig in enumerate(class_ids)}
    loader = DataLoader(dataset, batch_size=batch, shuffle=False, num_workers=2)
    ok = tot = 0
    for imgs, lbls in loader:
        y = torch.tensor([cont[l.item()] for l in lbls], device=device)
        x = _encode_imgs(imgs, device)
        logits = (x @ Z0.T) * model.logit_scale.exp()
        pred = logits.argmax(1)
        ok += (pred == y).sum().item(); tot += y.size(0)
    acc = ok / max(1, tot)
    print(f"[{name}] Acc={acc*100:.2f}%")
    return acc

@torch.no_grad()
def eval_head_only(dataset, class_ids, Z0, Gc, head, batch=128, name="HEAD", device="cuda"):
    cont = {orig:i for i, orig in enumerate(class_ids)}
    loader = DataLoader(dataset, batch_size=batch, shuffle=False, num_workers=2)
    ok = tot = 0
    for imgs, lbls in loader:
        y = torch.tensor([cont[l.item()] for l in lbls], device=device)
        x = _encode_imgs(imgs, device)
        logits, _, _, _, _ = head(x, Z0, Gc)
        pred = logits.argmax(1)
        ok += (pred == y).sum().item(); tot += y.size(0)
    acc = ok / max(1, tot)
    print(f"[{name}] Acc={acc*100:.2f}%")
    return acc

@torch.no_grad()
def compare_agreement(dataset, class_ids, Z0, Gc, head, batch=128, name="Compare", device="cuda"):
    cont = {orig:i for i, orig in enumerate(class_ids)}
    loader = DataLoader(dataset, batch_size=batch, shuffle=False, num_workers=2)
    agree = head_wins = zs_wins = tot = 0
    for imgs, lbls in loader:
        y = torch.tensor([cont[l.item()] for l in lbls], device=device)
        x = _encode_imgs(imgs, device)
        zs = (x @ Z0.T) * model.logit_scale.exp()
        hd, _, _, _, _ = head(x, Z0, Gc)
        p_zs = zs.argmax(1); p_hd = hd.argmax(1)
        agree += (p_zs == p_hd).sum().item()
        head_wins += ((p_hd == y) & (p_zs != y)).sum().item()
        zs_wins   += ((p_zs == y) & (p_hd != y)).sum().item()
        tot += y.size(0)
    print(f"[{name}] agree={agree/tot*100:.1f}% | head>zs={head_wins} | zs>head={zs_wins} | total={tot}")

print("== BASE ==")
acc_zs_base = eval_zs(test_base,  base_classes,  Z0_base,  name="ZS-Base",   device=device)
acc_head_base = eval_head_only(test_base, base_classes, Z0_base, Gc_base_mix, head, name="HEAD-Base", device=device)
compare_agreement(test_base, base_classes, Z0_base, Gc_base_mix, head, name="Base agreement", device=device)

print("\n== NOVEL ==")
acc_zs_novel = eval_zs(test_novel,  novel_classes,  Z0_novel,  name="ZS-Novel",   device=device)
acc_head_novel = eval_head_only(test_novel, novel_classes, Z0_novel, Gc_novel_mix, head, name="HEAD-Novel", device=device)
compare_agreement(test_novel, novel_classes, Z0_novel, Gc_novel_mix, head, name="Novel agreement", device=device)

== BASE ==
[ZS-Base] Acc=71.29%
[HEAD-Base] Acc=90.25%
[Base agreement] agree=72.0% | head>zs=551 | zs>head=82 | total=2473

== NOVEL ==
[ZS-Novel] Acc=78.24%
[HEAD-Novel] Acc=76.82%
[Novel agreement] agree=90.8% | head>zs=51 | zs>head=103 | total=3676
